# Processing the data

In [1]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

/Users/anubhavmaity/mambaforge/envs/transformers-course/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = 'bert-base-uncased'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
sequences = [
    'Trust yourself. You know more than you think you do.',
    'The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.'
]

In [6]:
batch = tokenizer(sequences, truncation=True, padding=True, return_tensors='pt')

In [8]:
batch['labels'] = torch.tensor([1, 1])

In [9]:
batch

{'input_ids': tensor([[  101,  3404,  4426,  1012,  2017,  2113,  2062,  2084,  2017,  2228,
          2017,  2079,  1012,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [  101,  1996,  2711,  1010,  2022,  2009, 10170,  2030,  3203,  1010,
          2040,  2038,  2025,  5165,  1999,  1037,  2204,  3117,  1010,  2442,
          2022,  2046,  3917,  8231,  5236,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]]), 'labels': tensor([1, 1])}

In [10]:
optimizer = AdamW(model.parameters())

/Users/anubhavmaity/mambaforge/envs/transformers-course/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
loss = model(**batch).loss

In [14]:
loss.backward()

In [15]:
optimizer.step()

In [16]:
model(**batch).loss

tensor(0.0957, grad_fn=<NllLossBackward0>)

## Loading a dataset from the hub

In [19]:
from datasets import load_dataset

In [20]:
raw_datasets = load_dataset('glue', 'mrpc')

Generating test split: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1725/1725 [00:00<00:00, 11435.83 examples/s]


In [21]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [22]:
raw_train_dataset = raw_datasets["train"]

In [26]:
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [24]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [34]:
raw_datasets["train"][15]

{'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
 'label': 0,
 'idx': 16}

In [35]:
raw_datasets["validation"][87]

{'sentence1': 'However , EPA officials would not confirm the 20 percent figure .',
 'sentence2': 'Only in the past few weeks have officials settled on the 20 percent figure .',
 'label': 0,
 'idx': 812}